# Introduction

This notebook explores using GPT-4 and GPT-3.5-Turbo for custom data quality tests, using the criteria used to identify [Data Grid datasets](https://centre.humdata.org/introducing-the-hdx-data-grid-a-way-to-find-and-fill-data-gaps/) on the fantastic [Humanitarian Data Exchange](https://data.humdata.org/) (HDX) Platform. 
See [this blog post]() for more details.

## Setup

1. For running OpenAI you will need to create a file called `api_key.txt` in the current directory and put your [OpenAI API key](https://beta.openai.com/account/api-keys) in there (just the API string, nothing else)
2. At time of writing submit an early access request for GPT-4 [here](https://openai.com/waitlist/gpt-4-api). You can also test with 'gpt-3.5-turbo' by setting the `model` variable below
3. Though the code below includes 

In [25]:
import numpy as np

import pandas as pd
import json
import os
import chardet
import shutil
import sys
import re
import traceback
import time

import numpy as np
from IPython.display import display, Markdown, Latex

import openai as ai
from openai import cli

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

pd.set_option("display.max_colwidth", None)

# File with HDX categories, as extracted from this document: https://data.humdata.org/dataset/2048a947-5714-4220-905b-e662cbcd14c8/resource/9d4121c6-b32b-4eb8-a707-209c79241970/download/state-of-open-humanitarian-data-2023.pdf
hdx_data_categories_file = "./data/HDX Data Grid Categories.csv"

# From HDX: https://data.humdata.org/dataset/kenya-production-of-rice-in-irrigation-schemes
irrigation_sheet="./data/number-of-acreage-under-irrigation.xlsx"

# From HDX: https://data.humdata.org/dataset/wfp-food-prices-for-chad
wfp_food_prices="./data/wfp_food_prices_tcd.csv"

# Sample of 200 Data Grid HDX datasets, with table excerpts from all tabular resources in the datase
# Code to produce this file can be happily provided on request to Medium author, but left out here to avoid 
# too much traffic on HDX
hdx_excerpts_file = "./data/datasets_excerpts.pkl"

# Open AI API key should be put into this file
ai.api_key_path = "./api_key.txt"

output_folder = "./data/"


## Analysis

First, let's identify datasets which were approved for DataGrid by checking the [data grid recipes](https://github.com/OCHA-DAP/data-grid-recipes) repo.

Read in Data Grid categories file ...

In [10]:
dg_categories = pd.read_csv(hdx_data_categories_file)
dg_categories = dg_categories[
    ["Category", "Subcategory", "Definition", "Datagrid recipe category"]
]
display(dg_categories)

,Category,Subcategory,Definition,Datagrid recipe category
0,Affected People,Internally Displaced Persons,Tabular data of the number of displaced people by location. Locations can be administrative divisions or other locations (such as camps) if an additional dataset defining those locations is also available.,IDPs
1,Affected People,Refugees and Persons of Concern,Tabular data of the number of refugees and persons of concern either in the country or originating from the country disaggregated by their current location. Locations can be administrative divisions or other locations (such as camps) if an additional dataset defining those locations is also available or if the locations' coordinates are defined in the tabular data.,REFUGEES POCs
2,Affected People,Returnees,Tabular data of the number of displaced people who have returned.,RETURNEES
3,Affected People,Humanitarian Needs,Tabular data of the number of people in need of humanitarian assistance by location and humanitarian cluster/sector.,HNO
4,Coordination & Context,3W - Who is doing what where,"List of organisations working on humanitarian issues, by humanitarian cluster/sector and disaggregated by administrative division.\n\nNote: An exception for the subnational rule is made for the IATI dataset which, if available, should always be included as an incomplete d",WHO WHAT WHERE
5,Coordination & Context,Funding,Tabular data listing the amount of funding provided by humanitarian cluster/sector.,FUNDING
6,Coordination & Context,Conflict Events,"Vector data or tabular data with coordinates describing the location, date, and type of conflict event.",CONFLICT EVENTS
7,Coordination & Context,Humanitarian Access,"Tabular or vector data describing the location of natural hazards, permissions, active fighting, or other access constraints that impact the delivery of humanitarian interventions.",HUMANITARIAN ACCESS
8,Coordination & Context,Climate Impact,"Tabular or vector data containing current and historical impacts of climate events relating to floods, droughts and storms. The data should specify the location of the event, date of the event, and contain at least one indicator of impact such as spatial extent of event, disruption to affected populations, destroyed infrastructure, and/or affected vegetation.",CLIMATE IMPACT
9,Food Security & Nutrition,Food Security,Vector data representing the IPC/CH acute food insecurity phase classification or tabular data representing population or percentage of population by IPC/CH phase and administrative division.,FOOD SECURITY


## Prompting GPT

Let's make it easier on the model by converting our categories information into text we can put in a prompt. GPT-4 will parse tables pretty well, but anything to reduce ambiguity is good ...

In [11]:
dg_categories["prompt_text"] = dg_categories.apply(
    lambda x: f"- Category '{x['Category']} : {x['Subcategory']}' is defined as: {x['Definition']}",
    axis=1,
)

category_prompt_text = dg_categories["prompt_text"].to_string(index=False, header=False)
display(category_prompt_text)

'                                                                                                                                                                                                                   - Category \'Affected People : Internally Displaced Persons\' is defined as: Tabular data of the number of displaced people by location. Locations can be administrative divisions or other locations (such as camps) if an additional dataset defining those locations is also available.\n                                              - Category \'Affected People : Refugees and Persons of Concern\' is defined as: Tabular data of the number of refugees and persons of concern either in the country or originating from the country disaggregated by their current location. Locations can be administrative divisions or other locations (such as camps) if an additional dataset defining those locations is also available or if the locations\' coordinates are defined in the tabular data.\n        

In [12]:
df = pd.read_excel(irrigation_sheet, sheet_name="Sheet1")
df = df.fillna("")
display(df)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Table 3: Number of acreage under irrigation,,,,,,,,,,,
1,,,OVERALL,,Sub county,,,,,,,
2,,,,,Chepalungu,,,,Bomet Central,,,
3,,,,,Male,,Female,,Male,,Female,
4,,,N,%,N,%,N,%,N,%,N,%
5,What is the average size of land you own that is currently under irrigation?,0 - 2 acres,22,2.8%,4,2.2%,10,3.8%,3,1.7%,5,2.9%
6,,2 - 5 acres,6,.8%,2,1.1%,2,.8%,0,0.0%,2,1.2%
7,,5 - 10 acres,1,.1%,0,0.0%,0,0.0%,0,0.0%,1,.6%
8,,More than 10 acres,0,0.0%,0,0.0%,0,0.0%,0,0.0%,0,0.0%
9,,None,760,96.3%,176,96.7%,251,95.4%,170,98.3%,163,95.3%


Typical table, with hierarchical columns and note in the first row. Let's convert it to a CSV string to be added in a prompt ...

In [13]:
csv_as_str = df[0:20].to_csv(index=False)
print(csv_as_str)

Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
Table 3: Number of acreage under irrigation,,,,,,,,,,,
,,OVERALL,,Sub county,,,,,,,
,,,,Chepalungu,,,,Bomet Central,,,
,,,,Male,,Female,,Male,,Female,
,,N,%,N,%,N,%,N,%,N,%
What is the average size of land you own that is currently under irrigation?,0 - 2 acres,22,2.8%,4,2.2%,10,3.8%,3,1.7%,5,2.9%
,2 - 5 acres,6,.8%,2,1.1%,2,.8%,0,0.0%,2,1.2%
,5 - 10 acres,1,.1%,0,0.0%,0,0.0%,0,0.0%,1,.6%
,More than 10 acres,0,0.0%,0,0.0%,0,0.0%,0,0.0%,0,0.0%
,None,760,96.3%,176,96.7%,251,95.4%,170,98.3%,163,95.3%
,Total,789,100.0%,182,100.0%,263,100.0%,173,100.0%,171,100.0%



In [ ]:
def prompt_model(prompts, temperature=0.0, model="gpt-4"):
    messages = [{"role": "system", "content": "You are a helpful assistant."}]
    for prompt in prompts:
        messages.append({"role": "user", "content": prompt})
        response = ai.ChatCompletion.create(
            model=model, temperature=temperature, messages=messages
        )
    return response["choices"][0]["message"]["content"]


prompts = []
prompts.append(
    f"Here is a list of HDX data categories with their definition: \n\n {category_prompt_text} \n\n"
)
prompts.append(
    f"Does the following table from file {irrigation_sheet} fall into one of the categories provided, if not say no. "
    f"If it does, which category and explain why? \n\n {csv_as_str} \n\n"
)

for model in ["gpt-3.5-turbo", "gpt-4"]:
    response = prompt_model(prompts, temperature=0.0, model=model)
    print(f"\n{model} Model response: \n\n{response}")

Good! Both correct! And importantly they predicted a negative case. This is important for our use-case.

Let's try with a table which is in Data Grid, Chad food prices.

In [7]:
df = pd.read_csv(wfp_food_prices)
df = df.fillna("")
csv_as_str = df[0:20].to_csv(index=False)

In [17]:
print(csv_as_str)

prompts = []
prompts.append(
    f"Here is a list of HDX data categories with their definition: \n\n {category_prompt_text} \n\n"
)
prompts.append(
    f"Does the following table from file {wfp_food_prices} fall into one of the categories provided, if not say no. "
    f"If it does, which category and explain why? \n\n {csv_as_str} \n\n"
)

for model in ["gpt-3.5-turbo", "gpt-4"]:
    response = prompt_model(prompts, temperature=0.0, model=model)
    print(f"\n{model} Model response: \n\n{response}")

date,admin1,admin2,market,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice
#date,#adm1+name,#adm2+name,#loc+market+name,#geo+lat,#geo+lon,#item+type,#item+name,#item+unit,#item+price+flag,#item+price+type,#currency,#value,#value+usd
2003-10-15,Barh El Gazal,Barh El Gazel Sud,Moussoro,13.640841,16.490069,cereals and tubers,Maize,KG,actual,Retail,XAF,134.0,0.2377
2003-10-15,Barh El Gazal,Barh El Gazel Sud,Moussoro,13.640841,16.490069,cereals and tubers,Millet,KG,actual,Retail,XAF,147.0,0.2608
2003-10-15,Lac,Mamdi,Bol,13.5,14.683333,cereals and tubers,Maize,KG,actual,Retail,XAF,81.0,0.1437
2003-10-15,Lac,Mamdi,Bol,13.5,14.683333,cereals and tubers,Maize (white),KG,actual,Retail,XAF,81.0,0.1437
2003-10-15,Logone Occidental,Lac Wey,Moundou,8.5666667,16.0833333,cereals and tubers,Millet,KG,actual,Retail,XAF,95.0,0.1685
2003-10-15,Logone Occidental,Lac Wey,Moundou,8.5666667,16.0833333,cereals and tubers,Sorghum,KG,actual,Retail,XAF,62.0,0.11
2003-10-15,Lo

Again, GPT-3.5-turbo was wrong as 'Food Prices' is not an accepted category, GPT-4 correct. Let's try with a few more files, including some which are in DataGrid, to see how well we do.

# Using multiple table excerpts and running for more datasets
OK, now let's loop through the Data Grid datasets, which should all be in an approved category.


In [86]:
def predict(data_excerpts, main_prompt, temperature=0.0):
    results = []
    for index, row in data_excerpts.iterrows():
        dataset = row["name"]
        title = row["title"]
        print(
            f"\n========================================= {dataset} =============================================\n"
        )

        prompts = []

        # Start the prompt by defining the categories we want to assign
        prompts.append(
            f"Here is a list of HDX data categories with their definition: \n\n {category_prompt_text} \n\n"
        )
        prompts.append(
            f"Here are excerpts from all the tables in this dataset: {title} ...\n\n"
        )

        # Build multiple prompts for each table excerpt for this dataset
        tables = row["table_excerpts"]
        for table in tables:
            table = json.loads(table)
            csv_as_str = table["table_excerpt"]
            sheet = table["sheet"]
            type = table["type"]
            filename = table["filename"]
            print(f"DATA > {filename} / {sheet}")
            prompts.append(
                f"Type {type} sheet {sheet} from file {filename} Table excerpt: \n\n {csv_as_str} \n\n"
            )

        # Finish up with our request
        prompts.append(main_prompt)

        actual_category = row["datagrid_category"]
        d = {
            "dataset_name": dataset,
            "filename": filename,
            "prompts": prompts,
            "actual_category": actual_category,
        }

        # Send our prompt array to two models
        for model in ["gpt-3.5-turbo", "gpt-4"]:
            # for model in ['gpt-3.5-turbo']:
            # GPT-4 is in test and can fail sometimes
            try:
                print(f"\nCalling model {model}")
                response = prompt_model(prompts, temperature=temperature, model=model)
                if "|" in response:
                    predicted_category = response.split("|")[1].strip()
                else:
                    predicted_category = response
                print(f"\n{model} Model response: \n\n{response}")
                match = actual_category == predicted_category
                d[f"{model}_response"] = response
                d[f"{model}_predicted_category"] = predicted_category
                d[f"{model}_match"] = match
                print(
                    f"******* RESULT: || {match} || prediced {predicted_category}, actual {actual_category} *******"
                )
            except Exception as e:
                print(e)
        results.append(d)

    results = pd.DataFrame(results)
    return results


data_excerpts = pd.read_pickle(hdx_excerpts_file)

data_excerpts = data_excerpts[data_excerpts["is_datagrid"] == True]

data_excerpts = data_excerpts.sample(min(150, data_excerpts.shape[0]), random_state=42)

main_prompt = (
            "Does the dataset fall into exactly one of the categories mentioned above, if not say no. "
            "If it does, add a pipe charatcter '|' before and after the top category and sub-category category and explain why it was chosen step-by-step.\n\n"
            "What is the second most likely category if you had to pick one (adding a ^ character either side)? \n\n"
)

results = predict(data_excerpts, main_prompt, temperature=0.0)
results.to_excel(f"{output_folder}/results.xlsx")

print("Done")


========================================= mozambique-attacks-on-aid-operations-education-health-and-protection =============================================

DATA > ./data/prompts/mozambique-attacks-on-aid-operations-education-health-and-protection/2020 SHCC Health Care Mozambique Data.xlsx / 2020 SHCC Mozambique Data

Calling model gpt-3.5-turbo

gpt-3.5-turbo Model response: 

|Affected People : Health Care| - The dataset falls under this category as it provides information on attacks on health care workers and facilities in Mozambique. The table excerpt shows data on the number of health workers killed, kidnapped, arrested, threatened, injured, and sexually assaulted, as well as the number of attacks on facilities that reported destruction, damage, and armed entry. 

^|Coordination & Context : Conflict Events| - The dataset could also fall under this category as it provides information on attacks on aid operations and health care in the context of conflict in Mozambique. However, t

2023-03-26 20:38:16.402 | INFO     | openai.util:log_info:67 - error_code=context_length_exceeded error_message="This model's maximum context length is 4097 tokens. However, your messages resulted in 4152 tokens. Please reduce the length of the messages." error_param=messages error_type=invalid_request_error message='OpenAI API error received' stream_error=False


This model's maximum context length is 4097 tokens. However, your messages resulted in 4152 tokens. Please reduce the length of the messages.

Calling model gpt-4

gpt-4 Model response: 

Yes, the dataset falls into exactly one of the categories mentioned above. The most suitable category for this dataset is:

|Population & Socio-economy : Poverty Rate|

The dataset contains information about poverty rates in Colombia at a subnational level, including the Multidimensional Poverty Index (MPI), headcount ratios, and other related indicators. This category is chosen because it specifically focuses on poverty rates and their disaggregation by administrative divisions.

The second most likely category would be:

^Affected People : Humanitarian Needs^

This category could be considered because the dataset provides information on the needs of the population in terms of poverty and deprivation, which can be relevant for humanitarian assistance planning and interventions. However, the primary f

2023-03-26 20:46:17.367 | INFO     | openai.util:log_info:67 - error_code=context_length_exceeded error_message="This model's maximum context length is 4097 tokens. However, your messages resulted in 4185 tokens. Please reduce the length of the messages." error_param=messages error_type=invalid_request_error message='OpenAI API error received' stream_error=False


This model's maximum context length is 4097 tokens. However, your messages resulted in 4185 tokens. Please reduce the length of the messages.

Calling model gpt-4

gpt-4 Model response: 

Yes, the dataset falls into exactly one of the categories mentioned above.

|Population & Socio-economy : Poverty Rate|

This category was chosen because the dataset provides information on the Multidimensional Poverty Index (MPI) for subnational regions in the Democratic Republic of the Congo. The MPI is a measure of poverty that takes into account multiple dimensions, such as health, education, and living standards. The dataset includes information on the proportion of people who are MPI poor and experience deprivations in each of the indicators by subnational regions, as well as the contribution of deprivations to the MPI.

^Coordination & Context : Humanitarian Needs^

The second most likely category would be Coordination & Context : Humanitarian Needs, as the dataset provides information on the p

...

This model's maximum context length is 4097 tokens. However, your messages resulted in 4410 tokens. Please reduce the length of the messages.

Calling model gpt-4

gpt-4 Model response: 

Yes, the dataset falls into exactly one of the categories mentioned above. The most suitable category for this dataset is:

|Population & Socio-economy : Poverty Rate|

The dataset contains information on poverty rates in Bangladesh, specifically the Multidimensional Poverty Index (MPI) for subnational regions. It provides data on the proportion of people who are MPI poor and experience deprivations in various indicators such as health, education, and living standards. The dataset also includes information on the total population, population share by region, and the number of MPI poor by region.

The second most likely category, if I had to pick one, would be:

^Food Security & Nutrition : Food Security^

Although the dataset does not directly provide data on food security, the Multidimensional Povert

2023-03-27 04:03:58.719 | INFO     | openai.util:log_info:67 - error_code=context_length_exceeded error_message="This model's maximum context length is 4097 tokens. However, your messages resulted in 7032 tokens. Please reduce the length of the messages." error_param=messages error_type=invalid_request_error message='OpenAI API error received' stream_error=False


This model's maximum context length is 4097 tokens. However, your messages resulted in 7032 tokens. Please reduce the length of the messages.

Calling model gpt-4


2023-03-27 04:07:27.440 | INFO     | openai.util:log_info:67 - error_code=context_length_exceeded error_message="This model's maximum context length is 8192 tokens. However, your messages resulted in 16748 tokens. Please reduce the length of the messages." error_param=messages error_type=invalid_request_error message='OpenAI API error received' stream_error=False


This model's maximum context length is 8192 tokens. However, your messages resulted in 16748 tokens. Please reduce the length of the messages.
Done


In [87]:
def output_prediction_metrics(
    results, prediction_field="predicted_post_processed", actual_field="actual_category"
):
    """
    Prints out model performance report if provided results in the format:

    [
        {
            'prompt': ' \'ISO3\' | "[\'RWA\', \'RWA\', \'RWA\', \'RWA\', \'RWA\', \'RWA\', \'RWA\', \'RWA\']"',
            'predicted': ' #country+code+iso3+v_iso3+',
            'predicted_post_processed': '#country+code',
            'expected': '#country+code'
        },
        ... etc ...
    ]

    Parameters
    ----------
    results : list
        See above for format
    prediction_field : str
        Field name of element with prediction. Handy for comparing raw and post-processed predictions.
    """
    y_test = []
    y_pred = []
    for index, r in results.iterrows():
        if actual_field not in r:
            print("Provided results do not contain expected values.")
            sys.exit()
        y_pred.append(r[prediction_field])
        y_test.append(r[actual_field])

    print(f"Results for {prediction_field}, {len(results)} predictions ...\n")
    print(f"Accuracy: {round(accuracy_score(y_test, y_pred),2)}")
    print(
        f"Precision: {round(precision_score(y_test, y_pred, average='weighted', zero_division=0),2)}"
    )
    print(
        f"Recall: {round(recall_score(y_test, y_pred, average='weighted', zero_division=0),2)}"
    )
    print(
        f"F1: {round(f1_score(y_test, y_pred, average='weighted', zero_division=0),2)}"
    )

    return


results.fillna("", inplace=True)
results = results.drop_duplicates(subset=["dataset_name"])


def split_subcategory(category):
    if ":" in category:
        return category.split(":")[0].strip()
    else:
        return category


for c in [
    "gpt-3.5-turbo_predicted_category",
    "gpt-4_predicted_category",
    "actual_category",
]:
    results[f"{c}_1"] = results[c].apply(lambda x: split_subcategory(x))

for prediction_field in [
    "gpt-3.5-turbo_predicted_category_1",
    "gpt-4_predicted_category_1",
    "gpt-3.5-turbo_predicted_category",
    "gpt-4_predicted_category",
]:

    print(f"======> \n{prediction_field} ...")
    actual_category = "actual_category"
    if "_1" in prediction_field:
        actual_category += "_1"
    output_prediction_metrics(
        results.loc[
            (results["gpt-3.5-turbo_predicted_category"] != "")
            & (results["gpt-4_predicted_category"] != "")
        ],
        prediction_field=prediction_field,
        actual_field=actual_category,
    )

results.to_excel(f"{output_folder}/results_all.xlsx")

======> 
gpt-3.5-turbo_predicted_category_1 ...
Results for gpt-3.5-turbo_predicted_category_1, 53 predictions ...

Accuracy: 0.66
Precision: 0.86
Recall: 0.66
F1: 0.68
======> 
gpt-4_predicted_category_1 ...
Results for gpt-4_predicted_category_1, 53 predictions ...

Accuracy: 0.96
Precision: 0.97
Recall: 0.96
F1: 0.96
======> 
gpt-3.5-turbo_predicted_category ...
Results for gpt-3.5-turbo_predicted_category, 53 predictions ...

Accuracy: 0.57
Precision: 0.73
Recall: 0.57
F1: 0.6
======> 
gpt-4_predicted_category ...
Results for gpt-4_predicted_category, 53 predictions ...

Accuracy: 0.89
Precision: 0.92
Recall: 0.89
F1: 0.89


Let's look at times when GPT-4 didn't predict the category correctly ...

In [92]:
df = results.loc[results["gpt-4_match"] == False]

for index, row in df.iterrows():
    response = row["gpt-4_response"]
    predicted_second_category = response.split("^")[1].strip()
    print(f"Dataset: {row['dataset_name']}")
    # print(f"Dataset: {row['filename']}")
    print("")
    print(f"Actual: {row['actual_category']}")
    print(f"Predicted category: {row['gpt-4_predicted_category']}")
    print(f"Predicted second category: {predicted_second_category}\n")
    print(
        f"Secondary category matched: {predicted_second_category == row['actual_category']}"
    )
    print("=====================================================")

Dataset: mozambique-attacks-on-aid-operations-education-health-and-protection

Actual: Coordination & Context : Humanitarian Access
Predicted category: Coordination & Context : Conflict Events
Predicted second category: Health & Education : Health Facilities

Secondary category matched: False
Dataset: iraq-violence-against-civilians-and-vital-civilian-facilities

Actual: Coordination & Context : Humanitarian Access
Predicted category: Coordination & Context : Conflict Events
Predicted second category: Affected People : Humanitarian Needs

Secondary category matched: False
Dataset: south-sudan-access-incidents

Actual: Coordination & Context : Conflict Events
Predicted category: Coordination & Context : Humanitarian Access
Predicted second category: Coordination & Context : Conflict Events

Secondary category matched: True
Dataset: somalia-displacement-idps-returnees-baseline-assessment-iom-dtm

Actual: Affected People : Returnees
Predicted category: Affected People : Internally Displac

We can see in the above that in many cases GPT-4's second choice was the correct one. More than that, some datasets seem to contain two categories of information, in fact on the HDX they are used in multiple places at times. The problem is a bit more nuanced than presented here, for a future study we might instead take categories and predict datasets rather than the other way around.
